In [ ]:
import sys
sys.path.insert(0, "..")

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IA.utils import mapping, parameter_range
from scipy import stats
import math
import swifter
import matplotlib.pyplot as plt
plt.style.reload_library()
plt.style.use(['science'])

In [ ]:
!ls -lah analysis/not_uploaded/IA/ | grep scores-one
!ls -lah analysis/not_uploaded/IA/ | grep scores-three

In [ ]:
df_one = pd.read_csv("analysis/not_uploaded/IA/.scratch.ckpts.IA.pexels.scores-one.epoch-13.pth.txt")
df_one_class = pd.read_csv("analysis/not_uploaded/IA/.scratch.ckpts.IA.pexels.scores-one.change_class.epoch-5.pth.txt")[["img", "distortion", "level", "score"]]
df_one_regr = pd.read_csv("analysis/not_uploaded/IA/.scratch.ckpts.IA.pexels.scores-one.change_regress.epoch-9.pth.txt")[["img", "distortion", "level", "score"]]

In [ ]:
for df in [df_one, df_one_class, df_one_regr]:
    for col in df.columns:
        if not "score" in col:
            continue

        df[col] = df[col].swifter.apply(lambda row: eval(row)[0])

df_one = df_one.rename(columns={"score": "score_one"})
df_one_class = df_one_class.rename(columns={"score": "score_class"})
df_one_regr = df_one_regr.rename(columns={"score": "score_regr"})

df_one = df_one.set_index(["img", "distortion", "level"])
df_one_class = df_one_class.set_index(["img", "distortion", "level"])
df_one_regr = df_one_regr.set_index(["img", "distortion", "level"])

df = df_one.join([df_one_class, df_one_regr])
df = df.drop_duplicates()
df = df.dropna()

del df_one
del df_one_class
del df_one_regr

df = df.reset_index()
df

In [ ]:
df["intensity"] = [float("nan")]*len(df)
df.loc[df["level"] == 0, "intensity"] = 0
for t in ["styles", "technical", "composition"]:
    for dist in mapping[t].keys():
        for pol in mapping[t][dist].keys():
            if pol == "neg":
                for lvl, change in enumerate(reversed(mapping[t][dist][pol])):
                    df.loc[(df["distortion"] == change.split(";")[0])&(df["level"] == float(change.split(";")[1])), "intensity"] = -(lvl+1)
            if pol == "pos":
                for lvl, change in enumerate(mapping[t][dist][pol]):
                    df.loc[(df["distortion"] == change.split(";")[0])&(df["level"] == float(change.split(";")[1])), "intensity"] = lvl+1
df["intensity"] = df["intensity"].astype(int)
df

In [ ]:
sns.violinplot(data=df.rename(columns={"score_one":"score"}), x="intensity", y="score", color="steelblue")
plt.savefig("analysis/IA/score_one.pdf")

In [ ]:
sns.violinplot(data=df.rename(columns={"score_regr":"score"}), x="intensity", y="score", color="steelblue")
plt.savefig("analysis/IA/score_regr.pdf")

In [ ]:
sns.violinplot(data=df[df["distortion"].isin(mapping["styles"].keys())].rename(columns={"score_regr":"score"}), x="intensity", y="score", color="steelblue")
plt.savefig("analysis/IA/score_regr_styles.pdf")

In [ ]:
sns.violinplot(data=df[df["distortion"].isin(mapping["technical"].keys())].rename(columns={"score_regr":"score"}), x="intensity", y="score", color="steelblue")
plt.savefig("analysis/IA/score_regr_technical.pdf")

In [ ]:
sns.violinplot(data=df[df["distortion"].isin(mapping["composition"].keys())].rename(columns={"score_regr":"score"}), x="intensity", y="score", color="steelblue")
plt.savefig("analysis/IA/score_regr_composition.pdf")